# Ecoli Thames Data Processing

In [1]:
import os
import json
import pandas as pd
import bng_latlon as bl

### Specify the path(s) to the data

In [2]:
data_path = os.path.join("..", "data")

dir_new_samples_path = os.path.join(data_path, "Reader Output")
dir_store_path = os.path.join(data_path, "temporary results")
dir_further_features_path = os.path.join(data_path, "further_features")

manual_counting_path = os.path.join(
    dir_further_features_path, "Thames Sampling Ecoli - manual counting.xlsx"
)
water_quality_path = os.path.join(
    dir_further_features_path, "Thames Sampling Water Quality Data.xlsx"
)
historic_discharges_path = os.path.join(
    dir_further_features_path,
    "Thames Water Historic Discharges by site (missing fleet main before 2023).xlsx",
)

# file_old_samples_path = os.path.join(data_path, "historical_samples.csv")
file_old_samples_path = None
filename_new_samples = "merged_data.csv"

### Validate paths

In [3]:
if not os.path.isdir(dir_new_samples_path):
    raise ValueError(
        "The path to the folder containing the new samples excel files is not a directory"
    )

if not os.path.isdir(dir_store_path):
    raise ValueError(
        "The path to store the concatenated excel file is not a directory"
    )

if file_old_samples_path is not None:
    if not os.path.isfile(file_old_samples_path):
        raise ValueError(
            "The path to the file containing the old samples excel file is not a file"
        )

if not filename_new_samples.endswith(".csv"):
    raise ValueError("The name of the new file must include the csv extension")

## Load data

In [4]:
from genericpath import isdir


def append_data(new_data_folder, old_dataset_path):
    """
    Append the data in the new_data_folder to the old_dataset
    """

    new_samples = []
    for root, dirs, files in os.walk(new_data_folder):
        for file in files:
            if file.endswith(".xlsx"):
                new_samples.append(pd.read_excel(os.path.join(root, file)))
            elif file.endswith(".csv"):
                new_samples.append(pd.read_csv(os.path.join(root, file)))
            elif file.endswith(".DS_Store"):
                pass
            else:
                raise ValueError("The file extension is not supported")

        for dir in dirs:
            if isdir(os.path.join(root, dir)):
                new_samples.append(
                    append_data(os.path.join(root, dir), old_dataset_path)
                )

    new_samples_df = pd.concat(new_samples, ignore_index=True)
    if old_dataset_path is not None:
        if old_dataset_path.endswith(".xlsx"):
            old_samples_df = pd.read_excel(old_dataset_path)
        elif old_dataset_path.endswith(".csv"):
            old_samples_df = pd.read_csv(old_dataset_path)
        else:
            raise ValueError("The file extension is not supported")
        new_samples_df = pd.concat(
            [old_samples_df, new_samples_df], ignore_index=True
        )
    return new_samples_df

In [5]:
raw_reader_output_df = append_data(dir_new_samples_path, None)

In [6]:
raw_reader_output_df

,Technician,Sample ID,Image Date Time,Plate Type,Dilution,Barcode Text,Red With Gas Raw Count,Red With Gas Edited Count,Red With Gas Calculated Result,Red Without Gas Raw Count,Red Without Gas Edited Count,Red Without Gas Calculated Result,Blue With Gas Raw Count,Blue With Gas Edited Count,Blue With Gas Calculated Result,Blue Without Gas Raw Count,Blue Without Gas Edited Count,Blue Without Gas Calculated Result,Comments
0,Admin,20230905_1010_B1.1,10/13/2023 1:05:35 PM,ECCC,1:1,-,336,334,334,0,0,0,6,7,7,1,1,1,NaN
1,Admin,20230905_1010_B1.2,10/13/2023 1:06:12 PM,ECCC,1:1,-,365,364,364,0,0,0,8,9,9,0,0,0,NaN
2,Admin,20230905_1010_B2.1,10/13/2023 1:06:57 PM,ECCC,1:1,-,26,26,26,79,79,79,4,4,4,2,2,2,NaN
3,Admin,20230905_1010_B2.2,10/13/2023 1:07:50 PM,ECCC,1:1,-,43,43,43,107,107,107,10,10,10,3,2,2,NaN
4,Admin,20230905_1126_B1.1,10/13/2023 1:09:26 PM,ECCC,1:1,-,36,36,36,108,108,108,5,5,5,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3877,Admin,20230824_1405_D2.2,8/30/2023 2:41:47 PM,ECCC,1:1,-,60,52,52,120,128,128,15,15,15,2,2,2,NaN
3878,Admin,20230824_1420_E1.2,8/30/2023 2:43:18 PM,ECCC,1:1,-,30,25,25,91,96,96,6,6,6,3,3,3,NaN
3879,Admin,20230824_1420_E1.1,8/30/2023 2:45:21 PM,ECCC,1:1,-,45,34,34,84,95,95,9,9,9,1,1,1,NaN
3880,Admin,20230824_1420_E2.1,8/30/2023 2:46:30 PM,ECCC,1:1,-,58,49,49,93,102,102,4,4,4,1,1,1,NaN


In [7]:
raw_manual_counting_df = pd.read_excel(manual_counting_path)
raw_water_quality_df = pd.read_excel(water_quality_path)
raw_historic_discharges_df = pd.read_excel(
    historic_discharges_path, header=None
)

In [8]:
raw_manual_counting_df

,Counter,Date,Time,Site,Sample,Ecoli (1ml),Coliform (1ml)
0,Randa,2023-01-31,10:40:00,A,1.0,3.0,13.0
1,Randa,2023-01-31,10:40:00,A,2.0,3.0,7.0
2,Randa,2023-01-31,11:14:00,C,2.0,6.0,11.0
3,Randa,2023-01-31,11:15:00,C,1.0,8.0,11.0
4,Randa,2023-01-31,11:35:00,D,1.0,11.0,10.0
...,...,...,...,...,...,...,...
165,Alisha,2023-06-29,11:47:00,B,1.1,241.0,131.0
166,Alisha,2023-06-29,11:22:00,A,1.2,81.0,16.0
167,Alisha,2023-06-29,11:22:00,A,1.1,95.0,12.0
168,Alisha,2023-06-29,11:22:00,A,2.2,42.0,6.0


In [9]:
raw_water_quality_df

,FOLLOW UP,Sampler,Date,Time,Site,Sample,Temp C,Ph,Cond (ms)
0,NaN,Randa,2023-01-31,10:40:00,A,1,10.0,8.4,0.690000
1,NaN,Randa,2023-01-31,10:40:00,A,2,9.5,8.2,0.730000
2,NaN,Randa,2023-01-31,11:15:00,C,1,8.8,8.2,0.720000
3,NaN,Randa,2023-01-31,11:15:00,C,2,7.9,8.1,0.710000
4,X,Randa,2023-01-31,11:35:00,D,1,NaN,8.1,0.720000
...,...,...,...,...,...,...,...,...,...
691,NaN,NaN,2023-09-07,15:45:00,E,2,24.0,7.9,1.540000
692,X,NaN,2023-09-07,NaN,B,1,24.3,7.9,1.890000
693,X,NaN,2023-09-07,NaN,B,2,23.8,7.8,1.940000
694,X,NaN,2023-09-07,NaN,E,2,24.2,7.9,1.875000


In [10]:
raw_historic_discharges_df

,0
0,"{""meta"": {""publisher"": ""Thames Water Utilities..."
1,"{""meta"": {""publisher"": ""Thames Water Utilities..."
2,"{""meta"": {""publisher"": ""Thames Water Utilities..."
3,"{""meta"": {""publisher"": ""Thames Water Utilities..."
4,"{""meta"": {""publisher"": ""Thames Water Utilities..."
...,...
4171,"{""meta"": {""publisher"": ""Thames Water Utilities..."
4172,"{""meta"": {""publisher"": ""Thames Water Utilities..."
4173,"{""meta"": {""publisher"": ""Thames Water Utilities..."
4174,"{""meta"": {""publisher"": ""Thames Water Utilities..."


## Data Preprocessing

### Raw Automated Counting

In [11]:
reader_output_df = raw_reader_output_df.copy()

reader_output_df.drop_duplicates(inplace=True)

reader_output_df[["Date", "Time", "SiteSample"]] = raw_reader_output_df[
    "Sample ID"
].str.split("_", expand=True)

reader_output_df["Date"] = pd.to_datetime(
    reader_output_df["Date"], format="%Y%m%d"
).dt.date
reader_output_df["Time"].replace("XXXX", pd.NaT, inplace=True)  # type: ignore
reader_output_df["Time"] = pd.to_datetime(
    reader_output_df["Time"], format="%H%M", errors="coerce"
).dt.strftime("%H:%M")
reader_output_df["Image Date Time"] = pd.to_datetime(
    reader_output_df["Image Date Time"]
)

date_col = reader_output_df.pop("Date")
time_col = reader_output_df.pop("Time")

reader_output_df.insert(2, "Date", date_col)
reader_output_df.insert(3, "Time", time_col)

reader_output_df.insert(4, "Site", reader_output_df["SiteSample"].str.extract("([A-Za-z]+)", expand=True))  # type: ignore
reader_output_df.insert(5, "Sample", reader_output_df["SiteSample"].str.extract("([\d.]+)", expand=True))  # type: ignore

reader_output_df.drop(columns=["SiteSample"], inplace=True)
reader_output_df.drop(columns=["Sample ID"], inplace=True)

reader_output_df.drop(columns=["Barcode Text", "Plate Type"], inplace=True)

# drop useless columns
reader_output_df.drop(
    columns=[
        "Red With Gas Raw Count",
        "Red With Gas Edited Count",
        "Red Without Gas Raw Count",
        "Red Without Gas Edited Count",
        "Red Without Gas Calculated Result",
        "Blue With Gas Raw Count",
        "Blue With Gas Edited Count",
        "Blue Without Gas Raw Count",
        "Blue Without Gas Edited Count",
        "Blue Without Gas Calculated Result",
        "Comments",
    ],
    inplace=True,
)

reader_output_df.rename(
    {
        "Red With Gas Calculated Result": "Coliform (1ml)",
        "Blue With Gas Calculated Result": "Ecoli (1ml)",
    },
    axis=1,
    inplace=True,
)

reader_output_df["Sample"] = reader_output_df["Sample"].astype(float)
reader_output_df["Site"] = reader_output_df["Site"].str.upper()

reader_output_df.dropna(subset=["Coliform (1ml)", "Ecoli (1ml)"], inplace=True)


reader_output_df

<>:27: SyntaxWarning: invalid escape sequence '\d'
<>:27: SyntaxWarning: invalid escape sequence '\d'
/var/folders/z5/plf0_b5s39nb0_gctbxmpndc0000gn/T/ipykernel_94169/2257464726.py:27: SyntaxWarning: invalid escape sequence '\d'
  reader_output_df.insert(5, "Sample", reader_output_df["SiteSample"].str.extract("([\d.]+)", expand=True))  # type: ignore
/var/folders/z5/plf0_b5s39nb0_gctbxmpndc0000gn/T/ipykernel_94169/2257464726.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reader_output_df["Time"].replace("XXXX", pd.NaT, i

,Technician,Date,Time,Site,Sample,Image Date Time,Dilution,Coliform (1ml),Ecoli (1ml)
0,Admin,2023-09-05,10:10,B,1.1,2023-10-13 13:05:35,1:1,334,7
1,Admin,2023-09-05,10:10,B,1.2,2023-10-13 13:06:12,1:1,364,9
2,Admin,2023-09-05,10:10,B,2.1,2023-10-13 13:06:57,1:1,26,4
3,Admin,2023-09-05,10:10,B,2.2,2023-10-13 13:07:50,1:1,43,10
4,Admin,2023-09-05,11:26,B,1.1,2023-10-13 13:09:26,1:1,36,5
...,...,...,...,...,...,...,...,...,...
2169,Admin,2023-08-24,14:05,D,2.2,2023-08-30 14:41:47,1:1,52,15
2170,Admin,2023-08-24,14:20,E,1.2,2023-08-30 14:43:18,1:1,25,6
2171,Admin,2023-08-24,14:20,E,1.1,2023-08-30 14:45:21,1:1,34,9
2172,Admin,2023-08-24,14:20,E,2.1,2023-08-30 14:46:30,1:1,49,4


In [12]:
raw_extra_data_path = os.path.join(data_path, "2023_08_10_AM_pt2.csv")
raw_extra_data_df = pd.read_csv(raw_extra_data_path, sep=";")

In [13]:
extra_data_df = raw_extra_data_df.copy()

extra_data_df.drop_duplicates(inplace=True)

extra_data_df[["Date", "Time", "SiteSample"]] = raw_extra_data_df[
    "Sample ID"
].str.split("_", expand=True)

extra_data_df["Date"] = pd.to_datetime(
    extra_data_df["Date"], format="%Y%m%d"
).dt.date
extra_data_df["Time"].replace("XXXX", pd.NaT, inplace=True)  # type: ignore
extra_data_df["Time"] = pd.to_datetime(
    extra_data_df["Time"], format="%H%M", errors="coerce"
).dt.strftime("%H:%M")
extra_data_df["Image Date Time"] = pd.to_datetime(
    extra_data_df["Image Date Time"]
)

date_col = extra_data_df.pop("Date")
time_col = extra_data_df.pop("Time")

extra_data_df.insert(2, "Date", date_col)
extra_data_df.insert(3, "Time", time_col)

extra_data_df.insert(4, "Site", extra_data_df["SiteSample"].str.extract("([A-Za-z]+)", expand=True))  # type: ignore
extra_data_df.insert(5, "Sample", extra_data_df["SiteSample"].str.extract("([\d.]+)", expand=True))  # type: ignore

extra_data_df.drop(columns=["SiteSample"], inplace=True)
extra_data_df.drop(columns=["Sample ID"], inplace=True)
extra_data_df.drop(columns=["Plate Type"], inplace=True)

extra_data_df["Sample"] = extra_data_df["Sample"].astype(float)
extra_data_df["Site"] = extra_data_df["Site"].str.upper()

extra_data_df.dropna(subset=["Coliform (1ml)", "Ecoli (1ml)"], inplace=True)

<>:27: SyntaxWarning: invalid escape sequence '\d'
<>:27: SyntaxWarning: invalid escape sequence '\d'
/var/folders/z5/plf0_b5s39nb0_gctbxmpndc0000gn/T/ipykernel_94169/1329072125.py:27: SyntaxWarning: invalid escape sequence '\d'
  extra_data_df.insert(5, "Sample", extra_data_df["SiteSample"].str.extract("([\d.]+)", expand=True))  # type: ignore
/var/folders/z5/plf0_b5s39nb0_gctbxmpndc0000gn/T/ipykernel_94169/1329072125.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  extra_data_df["Time"].replace("XXXX", pd.NaT, inplace=Tr

In [14]:
reader_output_df = pd.concat(
    [reader_output_df, extra_data_df], ignore_index=True
)
reader_output_df.drop_duplicates(inplace=True)

In [15]:
# Split the sample column into two columns (Bottle and Sample) for those samples that are in the format "Bottle.Sample"
reader_output_df.insert(loc=4, column="Bottle", value=reader_output_df["Sample"].astype(str).str.extract("(\d+)", expand=True))  # type: ignore
reader_output_df["Sample"] = reader_output_df["Sample"].astype(str).str.extract("(\d+$)", expand=True)  # type: ignore

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
/var/folders/z5/plf0_b5s39nb0_gctbxmpndc0000gn/T/ipykernel_94169/2380658007.py:2: SyntaxWarning: invalid escape sequence '\d'
  reader_output_df.insert(loc=4, column="Bottle", value=reader_output_df["Sample"].astype(str).str.extract("(\d+)", expand=True))  # type: ignore
/var/folders/z5/plf0_b5s39nb0_gctbxmpndc0000gn/T/ipykernel_94169/2380658007.py:3: SyntaxWarning: invalid escape sequence '\d'
  reader_output_df["Sample"] = reader_output_df["Sample"].astype(str).str.extract("(\d+$)", expand=True)  # type: ignore


In [16]:
reader_output_df

,Technician,Date,Time,Site,Bottle,Sample,Image Date Time,Dilution,Coliform (1ml),Ecoli (1ml)
0,Admin,2023-09-05,10:10,B,1,1,2023-10-13 13:05:35,1:1,334,7
1,Admin,2023-09-05,10:10,B,1,2,2023-10-13 13:06:12,1:1,364,9
2,Admin,2023-09-05,10:10,B,2,1,2023-10-13 13:06:57,1:1,26,4
3,Admin,2023-09-05,10:10,B,2,2,2023-10-13 13:07:50,1:1,43,10
4,Admin,2023-09-05,11:26,B,1,1,2023-10-13 13:09:26,1:1,36,5
...,...,...,...,...,...,...,...,...,...,...
2165,Vee,2023-08-10,10:40,C,1,1,2023-11-08 10:17:00,1:1,16,3
2166,Vee,2023-08-10,10:40,C,1,1,2023-11-08 10:17:00,1:1,17,3
2167,Vee,2023-08-10,10:25,B,2,2,2023-11-08 10:16:00,1:1,255,2
2168,Vee,2023-08-10,10:25,B,2,2,2023-11-08 10:16:00,1:1,236,2


In [17]:
print(
    f'Date range of the automatic couting: {reader_output_df["Date"].min()} - {reader_output_df["Date"].max()}'
)

Date range of the automatic couting: 2023-07-31 - 2023-09-07


In [18]:
reader_output_df["Sample"].unique()

array(['1', '2', '3', '4'], dtype=object)

### Raw Manual Counting

In [19]:
manual_counting_df = raw_manual_counting_df.copy()

manual_counting_df.drop_duplicates(inplace=True)

manual_counting_df["Sample"] = manual_counting_df["Sample"].astype(float)
manual_counting_df.rename(
    {"Counter": "Technician"},
    axis=1,
    inplace=True,
)

manual_counting_df["Date"] = manual_counting_df["Date"].dt.date
manual_counting_df["Time"] = pd.to_datetime(
    manual_counting_df["Time"], format="%H:%M:%S", errors="coerce"
).dt.strftime("%H:%M")

manual_counting_df.dropna(
    subset=["Coliform (1ml)", "Ecoli (1ml)"], inplace=True
)

In [20]:
# Split the sample column into two columns (Bottle and Sample) for those samples that are in the format "Bottle.Sample"
manual_counting_df.insert(loc=4, column="Bottle", value=manual_counting_df["Sample"].astype(str).str.extract("(\d+)", expand=True))  # type: ignore
manual_counting_df["Sample"] = manual_counting_df["Sample"].astype(str).str.extract("(\d+$)", expand=True)  # type: ignore

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
/var/folders/z5/plf0_b5s39nb0_gctbxmpndc0000gn/T/ipykernel_94169/2690319564.py:2: SyntaxWarning: invalid escape sequence '\d'
  manual_counting_df.insert(loc=4, column="Bottle", value=manual_counting_df["Sample"].astype(str).str.extract("(\d+)", expand=True))  # type: ignore
/var/folders/z5/plf0_b5s39nb0_gctbxmpndc0000gn/T/ipykernel_94169/2690319564.py:3: SyntaxWarning: invalid escape sequence '\d'
  manual_counting_df["Sample"] = manual_counting_df["Sample"].astype(str).str.extract("(\d+$)", expand=True)  # type: ignore


In [21]:
manual_counting_df

,Technician,Date,Time,Site,Bottle,Sample,Ecoli (1ml),Coliform (1ml)
0,Randa,2023-01-31,10:40,A,1,0,3.0,13.0
1,Randa,2023-01-31,10:40,A,2,0,3.0,7.0
2,Randa,2023-01-31,11:14,C,2,0,6.0,11.0
3,Randa,2023-01-31,11:15,C,1,0,8.0,11.0
4,Randa,2023-01-31,11:35,D,1,0,11.0,10.0
...,...,...,...,...,...,...,...,...
165,Alisha,2023-06-29,11:47,B,1,1,241.0,131.0
166,Alisha,2023-06-29,11:22,A,1,2,81.0,16.0
167,Alisha,2023-06-29,11:22,A,1,1,95.0,12.0
168,Alisha,2023-06-29,11:22,A,2,2,42.0,6.0


In [22]:
print(
    f'Date range of the manual counting dataset: {manual_counting_df["Date"].min()} - {manual_counting_df["Date"].max()}'
)

Date range of the manual counting dataset: 2023-01-31 - 2023-06-29


In [23]:
manual_counting_df["Sample"].unique()

array(['0', '1', '2', '3'], dtype=object)

### Raw Water Quality

In [24]:
water_quality_df = raw_water_quality_df.copy()

water_quality_df["Sample"] = water_quality_df["Sample"].astype(float)
water_quality_df.rename(
    {"Sampler": "Technician"},
    axis=1,
    inplace=True,
)

water_quality_df = water_quality_df[water_quality_df["FOLLOW UP"] != "X"]
water_quality_df["Date"] = water_quality_df["Date"].dt.date
water_quality_df["Time"] = pd.to_datetime(
    water_quality_df["Time"], format="%H:%M:%S", errors="coerce"
).dt.strftime("%H:%M")

water_quality_df.drop(columns=["FOLLOW UP"], inplace=True)
water_quality_df.dropna(subset=["Temp C", "Ph", "Cond (ms)"], inplace=True)

In [25]:
# Split the sample column into two columns (Bottle and Sample) for those samples that are in the format "Bottle.Sample"
water_quality_df.insert(loc=4, column="Bottle", value=water_quality_df["Sample"].astype(str).str.extract("(\d+)", expand=True))  # type: ignore
water_quality_df.drop(columns=["Sample"], inplace=True)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
/var/folders/z5/plf0_b5s39nb0_gctbxmpndc0000gn/T/ipykernel_94169/3854122545.py:2: SyntaxWarning: invalid escape sequence '\d'
  water_quality_df.insert(loc=4, column="Bottle", value=water_quality_df["Sample"].astype(str).str.extract("(\d+)", expand=True))  # type: ignore


In [26]:
water_quality_df

,Technician,Date,Time,Site,Bottle,Temp C,Ph,Cond (ms)
0,Randa,2023-01-31,10:40,A,1,10.0,8.40,0.69
1,Randa,2023-01-31,10:40,A,2,9.5,8.20,0.73
2,Randa,2023-01-31,11:15,C,1,8.8,8.20,0.72
3,Randa,2023-01-31,11:15,C,2,7.9,8.10,0.71
8,Randa,2023-01-31,11:55,E,1,8.1,7.97,0.79
...,...,...,...,...,...,...,...,...
687,NaN,2023-09-07,15:27,D,2,25.0,7.90,1.46
688,NaN,2023-09-07,15:32,A,1,27.7,8.00,1.86
689,NaN,2023-09-07,15:32,A,2,26.5,8.00,1.89
690,NaN,2023-09-07,15:45,E,1,25.0,7.80,1.52


In [27]:
print(
    f'Date range of the water quality: {water_quality_df["Date"].min()} - {water_quality_df["Date"].max()}'
)

Date range of the water quality: 2023-01-31 - 2023-09-07


## Merge Data

In [28]:
reader_manual_df = pd.concat(
    [reader_output_df, manual_counting_df], ignore_index=True
)

In [29]:
reader_manual_df

,Technician,Date,Time,Site,Bottle,Sample,Image Date Time,Dilution,Coliform (1ml),Ecoli (1ml)
0,Admin,2023-09-05,10:10,B,1,1,2023-10-13 13:05:35,1:1,334,7
1,Admin,2023-09-05,10:10,B,1,2,2023-10-13 13:06:12,1:1,364,9
2,Admin,2023-09-05,10:10,B,2,1,2023-10-13 13:06:57,1:1,26,4
3,Admin,2023-09-05,10:10,B,2,2,2023-10-13 13:07:50,1:1,43,10
4,Admin,2023-09-05,11:26,B,1,1,2023-10-13 13:09:26,1:1,36,5
...,...,...,...,...,...,...,...,...,...,...
2330,Alisha,2023-06-29,11:47,B,1,1,NaT,NaN,131.0,241.0
2331,Alisha,2023-06-29,11:22,A,1,2,NaT,NaN,16.0,81.0
2332,Alisha,2023-06-29,11:22,A,1,1,NaT,NaN,12.0,95.0
2333,Alisha,2023-06-29,11:22,A,2,2,NaT,NaN,6.0,42.0


In [30]:
print(
    f'Date range of the manual counting dataset: {reader_manual_df["Date"].min()} - {reader_manual_df["Date"].max()}'
)

Date range of the manual counting dataset: 2023-01-31 - 2023-09-07


In [31]:
full_df = pd.merge(
    left=reader_manual_df.rename({"Technician": "Technician Counting"}, axis=1),
    right=water_quality_df.rename(
        {"Technician": "Technician Water Quality"}, axis=1
    ),
    how="inner",
    on=["Date", "Time", "Site", "Bottle"],
)

In [32]:
full_df

,Technician Counting,Date,Time,Site,Bottle,Sample,Image Date Time,Dilution,Coliform (1ml),Ecoli (1ml),Technician Water Quality,Temp C,Ph,Cond (ms)
0,Admin,2023-09-05,10:10,B,1,1,2023-10-13 13:05:35,1:1,334,7,NaN,22.0,7.6,2.15625
1,Admin,2023-09-05,10:10,B,1,2,2023-10-13 13:06:12,1:1,364,9,NaN,22.0,7.6,2.15625
2,Admin,2023-09-05,10:10,B,2,1,2023-10-13 13:06:57,1:1,26,4,NaN,21.0,7.6,2.12500
3,Admin,2023-09-05,10:10,B,2,2,2023-10-13 13:07:50,1:1,43,10,NaN,21.0,7.6,2.12500
4,Admin,2023-09-05,11:26,B,1,1,2023-10-13 13:09:26,1:1,36,5,NaN,21.0,7.6,1.81250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,Alisha,2023-06-29,11:47,B,1,1,NaT,NaN,131.0,241.0,"Anan, Alisha",22.0,7.9,3.00000
1508,Alisha,2023-06-29,11:22,A,1,2,NaT,NaN,16.0,81.0,"Anan, Alisha",22.0,7.9,3.28125
1509,Alisha,2023-06-29,11:22,A,1,1,NaT,NaN,12.0,95.0,"Anan, Alisha",22.0,7.9,3.28125
1510,Alisha,2023-06-29,11:22,A,2,2,NaT,NaN,6.0,42.0,"Anan, Alisha",22.0,7.9,3.31250


In [33]:
full_df[
    full_df[["Coliform (1ml)", "Ecoli (1ml)", "Temp C", "Ph", "Cond (ms)"]]
    .notnull()
    .all(axis=1)
]

,Technician Counting,Date,Time,Site,Bottle,Sample,Image Date Time,Dilution,Coliform (1ml),Ecoli (1ml),Technician Water Quality,Temp C,Ph,Cond (ms)
0,Admin,2023-09-05,10:10,B,1,1,2023-10-13 13:05:35,1:1,334,7,NaN,22.0,7.6,2.15625
1,Admin,2023-09-05,10:10,B,1,2,2023-10-13 13:06:12,1:1,364,9,NaN,22.0,7.6,2.15625
2,Admin,2023-09-05,10:10,B,2,1,2023-10-13 13:06:57,1:1,26,4,NaN,21.0,7.6,2.12500
3,Admin,2023-09-05,10:10,B,2,2,2023-10-13 13:07:50,1:1,43,10,NaN,21.0,7.6,2.12500
4,Admin,2023-09-05,11:26,B,1,1,2023-10-13 13:09:26,1:1,36,5,NaN,21.0,7.6,1.81250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,Alisha,2023-06-29,11:47,B,1,1,NaT,NaN,131.0,241.0,"Anan, Alisha",22.0,7.9,3.00000
1508,Alisha,2023-06-29,11:22,A,1,2,NaT,NaN,16.0,81.0,"Anan, Alisha",22.0,7.9,3.28125
1509,Alisha,2023-06-29,11:22,A,1,1,NaT,NaN,12.0,95.0,"Anan, Alisha",22.0,7.9,3.28125
1510,Alisha,2023-06-29,11:22,A,2,2,NaT,NaN,6.0,42.0,"Anan, Alisha",22.0,7.9,3.31250


In [34]:
full_df["Technician Water Quality"] = full_df[
    "Technician Water Quality"
].str.replace(", ", " - ")

In [35]:
final_df = full_df.copy()

In [36]:
final_df["Time"] = final_df["Time"].fillna("00:00")

final_df.insert(
    loc=1,
    column="DateTime",
    value=pd.to_datetime(
        final_df["Date"].astype(str) + " " + final_df["Time"].astype(str)
    ),
)

final_df.drop(columns=["Date", "Time"], inplace=True)

# Build Final Dataset

In [37]:
final_df

,Technician Counting,DateTime,Site,Bottle,Sample,Image Date Time,Dilution,Coliform (1ml),Ecoli (1ml),Technician Water Quality,Temp C,Ph,Cond (ms)
0,Admin,2023-09-05 10:10:00,B,1,1,2023-10-13 13:05:35,1:1,334,7,NaN,22.0,7.6,2.15625
1,Admin,2023-09-05 10:10:00,B,1,2,2023-10-13 13:06:12,1:1,364,9,NaN,22.0,7.6,2.15625
2,Admin,2023-09-05 10:10:00,B,2,1,2023-10-13 13:06:57,1:1,26,4,NaN,21.0,7.6,2.12500
3,Admin,2023-09-05 10:10:00,B,2,2,2023-10-13 13:07:50,1:1,43,10,NaN,21.0,7.6,2.12500
4,Admin,2023-09-05 11:26:00,B,1,1,2023-10-13 13:09:26,1:1,36,5,NaN,21.0,7.6,1.81250
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,Alisha,2023-06-29 11:47:00,B,1,1,NaT,NaN,131.0,241.0,Anan - Alisha,22.0,7.9,3.00000
1508,Alisha,2023-06-29 11:22:00,A,1,2,NaT,NaN,16.0,81.0,Anan - Alisha,22.0,7.9,3.28125
1509,Alisha,2023-06-29 11:22:00,A,1,1,NaT,NaN,12.0,95.0,Anan - Alisha,22.0,7.9,3.28125
1510,Alisha,2023-06-29 11:22:00,A,2,2,NaT,NaN,6.0,42.0,Anan - Alisha,22.0,7.9,3.31250


In [129]:
# replace TNTC and 0 values with 'value'

value = 500

final_df["Coliform (1ml)"] = (
    final_df["Coliform (1ml)"].replace(["TNTC"], value).astype(float)
)
final_df["Coliform (1ml)"] = final_df["Coliform (1ml)"].replace(0, value)

final_df["Ecoli (1ml)"] = (
    final_df["Ecoli (1ml)"].replace("TNTC", value=value).astype(float)
)

In [130]:
# fix cond values
# some of the values are in ms and some in us
# we will convert all to us (divide by 1000)
# I take 200 as the threshold to decide if the value is in ms or us
# because I saw that the only values that are in ms are way above 200
# and the values that are in us are way below 200
final_df["Cond (ms)"] = final_df["Cond (ms)"].apply(
    lambda x: x / 1000 if x > 200 else x
)

## Overall Mean
Here a unique sample for each pair of DateTime and Site is built by averaging every value without keeping into consideration the Bottles and the Samples per Bottles.

In [131]:
# Average of values is computed DateTime-wise
overall_mean_df = (
    final_df.groupby(["DateTime", "Site"])
    .agg(
        {
            "Technician Water Quality": "first",
            "Technician Counting": "first",
            "Temp C": ["mean", "std"],
            "Ph": ["mean", "std"],
            "Cond (ms)": ["mean", "std"],
            "Coliform (1ml)": ["mean", "std"],
            "Ecoli (1ml)": ["mean", "std"],
        }
    )
    .reset_index()
)

In [132]:
overall_mean_df.columns = [
    "_".join(col) if col[1] == "mean" or col[1] == "std" else col[0]
    for col in overall_mean_df.columns.values
]

In [133]:
overall_mean_df.sort_values(by=["Site", "DateTime"], inplace=True)
overall_mean_df.reset_index(drop=True, inplace=True)

In [134]:
overall_mean_df

,DateTime,Site,Technician Water Quality,Technician Counting,Temp C_mean,Temp C_std,Ph_mean,Ph_std,Cond (ms)_mean,Cond (ms)_std,Coliform (1ml)_mean,Coliform (1ml)_std,Ecoli (1ml)_mean,Ecoli (1ml)_std
0,2023-01-31 10:40:00,A,Randa,Randa,9.75,0.353553,8.30,0.141421,0.7100,0.028284,10.00,4.242641,3.0,0.000000
1,2023-02-07 07:30:00,A,Randa,Randa,6.50,0.707107,7.80,0.141421,0.7450,0.007071,17.00,2.828427,6.5,0.707107
2,2023-02-14 09:00:00,A,Randa,Randa,8.50,0.707107,7.80,0.000000,0.8450,0.021213,23.50,3.535534,7.5,3.535534
3,2023-02-21 15:15:00,A,Randa,Randa,10.50,0.707107,7.95,0.070711,1.2900,0.028284,16.00,5.656854,6.0,1.414214
4,2023-02-22 11:50:00,A,Randa,Randa,12.00,NaN,7.90,NaN,0.7400,NaN,54.00,NaN,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,2023-01-31 12:13:00,F,Randa,Randa,8.10,0.282843,8.09,0.014142,0.7650,0.007071,6.00,2.828427,7.5,3.535534
263,2023-02-07 08:48:00,F,Randa,Randa,6.00,0.000000,8.10,0.000000,0.7850,0.007071,14.00,0.000000,8.0,2.828427
264,2023-02-14 10:25:00,F,Randa,Randa,8.50,0.707107,7.95,0.070711,0.7800,0.000000,12.50,3.535534,5.0,2.828427
265,2023-02-21 16:40:00,F,Randa,Randa,9.50,0.707107,8.00,0.141421,1.1400,0.028284,20.00,7.071068,5.0,0.000000


## Step-wise Mean
Here a unique sample for each pair of DateTime and Site is built by averaging first the automatic reader's outputs for each Sample of each Bottle, then a second average is done between Samples from the same Bottle, and finally the average of the Bottles for the same Site and DateTime is performed.

In [135]:
# First mean: Sample-wise average
sub_reader_mean_df = (
    final_df.groupby(
        ["DateTime", "Site", "Bottle", "Sample"],
    )
    .agg(
        {
            "Technician Water Quality": "first",
            "Technician Counting": "first",
            "Temp C": ["mean", "std"],
            "Ph": ["mean", "std"],
            "Cond (ms)": ["mean", "std"],
            "Coliform (1ml)": ["mean", "std"],
            "Ecoli (1ml)": ["mean", "std"],
        },
    )
    .reset_index()
)

In [136]:
sub_reader_mean_df.columns = [
    "_".join(col) if col[1] == "mean" or col[1] == "std" else col[0]
    for col in sub_reader_mean_df.columns.values
]

In [137]:
# Second mean: Bottle-wise average
sub_bottle_mean_df = (
    sub_reader_mean_df.groupby(
        ["DateTime", "Site", "Bottle"],
    )
    .agg(
        {
            "Technician Water Quality": "first",
            "Technician Counting": "first",
            "Temp C_mean": ["mean", "std"],
            "Ph_mean": ["mean", "std"],
            "Cond (ms)_mean": ["mean", "std"],
            "Coliform (1ml)_mean": ["mean", "std"],
            "Ecoli (1ml)_mean": ["mean", "std"],
        },
    )
    .reset_index()
)

In [138]:
sub_bottle_mean_df.columns = [
    "_".join(col) if col[1] == "mean" or col[1] == "std" else col[0]
    for col in sub_bottle_mean_df.columns.values
]

In [139]:
# Third mean: Site-wise average
sub_site_mean_df = (
    sub_bottle_mean_df.groupby(
        ["DateTime", "Site"],
    )
    .agg(
        {
            "Technician Water Quality": "first",
            "Technician Counting": "first",
            "Temp C_mean_mean": ["mean", "std"],
            "Ph_mean_mean": ["mean", "std"],
            "Cond (ms)_mean_mean": ["mean", "std"],
            "Coliform (1ml)_mean_mean": ["mean", "std"],
            "Ecoli (1ml)_mean_mean": ["mean", "std"],
        },
    )
    .reset_index()
)

In [140]:
sub_site_mean_df.columns = [
    "_".join(col) if col[1] == "mean" or col[1] == "std" else col[0]
    for col in sub_site_mean_df.columns.values
]

In [141]:
sub_site_mean_df.columns = [
    "_".join([col.split("_")[0], col.split("_")[-1]]) if "mean" in col else col
    for col in sub_site_mean_df.columns.values
]

In [142]:
sub_site_mean_df.sort_values(by=["Site", "DateTime"], inplace=True)
sub_site_mean_df.reset_index(drop=True, inplace=True)

In [143]:
sub_site_mean_df

,DateTime,Site,Technician Water Quality,Technician Counting,Temp C_mean,Temp C_std,Ph_mean,Ph_std,Cond (ms)_mean,Cond (ms)_std,Coliform (1ml)_mean,Coliform (1ml)_std,Ecoli (1ml)_mean,Ecoli (1ml)_std
0,2023-01-31 10:40:00,A,Randa,Randa,9.75,0.353553,8.30,0.141421,0.7100,0.028284,10.00,4.242641,3.0,0.000000
1,2023-02-07 07:30:00,A,Randa,Randa,6.50,0.707107,7.80,0.141421,0.7450,0.007071,17.00,2.828427,6.5,0.707107
2,2023-02-14 09:00:00,A,Randa,Randa,8.50,0.707107,7.80,0.000000,0.8450,0.021213,23.50,3.535534,7.5,3.535534
3,2023-02-21 15:15:00,A,Randa,Randa,10.50,0.707107,7.95,0.070711,1.2900,0.028284,16.00,5.656854,6.0,1.414214
4,2023-02-22 11:50:00,A,Randa,Randa,12.00,NaN,7.90,NaN,0.7400,NaN,54.00,NaN,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,2023-01-31 12:13:00,F,Randa,Randa,8.10,0.282843,8.09,0.014142,0.7650,0.007071,6.00,2.828427,7.5,3.535534
263,2023-02-07 08:48:00,F,Randa,Randa,6.00,0.000000,8.10,0.000000,0.7850,0.007071,14.00,0.000000,8.0,2.828427
264,2023-02-14 10:25:00,F,Randa,Randa,8.50,0.707107,7.95,0.070711,0.7800,0.000000,12.50,3.535534,5.0,2.828427
265,2023-02-21 16:40:00,F,Randa,Randa,9.50,0.707107,8.00,0.141421,1.1400,0.028284,20.00,7.071068,5.0,0.000000


In [144]:
overall_mean_df.to_excel(
    os.path.join(dir_store_path, "bacteria_dataset.xlsx"), index=False
)